In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
import os
os.chdir('..')

In [6]:
import os

import wandb
from omegaconf import OmegaConf

### Download locally the best models from wandb

---

In [4]:
CKPT_DIR = "/scratch/izar/cizinsky/rl-for-kinetics/best_models"

In [5]:
!mkdir -p {CKPT_DIR}

#### Get overview of the relevant runs

In [6]:
api = wandb.Api()

runs = api.runs("ludekcizinsky/rl-renaissance")

tagged_runs = [run for run in runs if "part2" in run.tags]

for run in tagged_runs:
    print(f"{run.id} | {run.name} | tags: {run.tags}")

kvxbygvv | zesty-bird-206 | tags: ['baseline', 'part2']
aockrykl | pious-pyramid-211 | tags: ['max_log_std', 'part2']
tg2aivow | feasible-terrain-212 | tags: ['max_log_std', 'part2']


#### Download the best model for each run

In [7]:
run_ids = [run.id for run in tagged_runs]

for run_id in ["aockrykl"]:
    print(f"Downloading checkpoint for run {run_id}...")
    run = next((run for run in runs if run.id == run_id), None)
    assert run is not None, "Run not found!"

    artifact_path = f"ludekcizinsky/rl-renaissance/{run.name}:v0"
    print(artifact_path)
    artifact = api.artifact(artifact_path, type="model")
    os.makedirs(f"{CKPT_DIR}/{run.name}", exist_ok=True)
    download_path = f"{CKPT_DIR}/{run.name}"

    # Config
    run_cfg = OmegaConf.create(run.config)
    OmegaConf.save(run_cfg, f"{download_path}/config.yaml")

    # Checkpoints
    artifact.download(download_path)
    print(f"Downloaded checkpoint to {download_path}.")

ludekcizinsky/rl-renaissance/pious-pyramid-211:v0


wandb:   4 of 4 files downloaded.  


Downloaded checkpoint to /scratch/izar/cizinsky/rl-for-kinetics/best_models/pious-pyramid-211.


### Inference

---

```bash
apptainer shell --nv --bind "$(pwd)":/home/renaissance/work --bind "/scratch/izar/$USER/rl-for-kinetics/output:/home/renaissance/output" --bind "/scratch/izar/$USER/rl-for-kinetics/best_models:/home/renaissance/best_models" /scratch/izar/$USER/images/renaissance_with_ml.sif
```

```bash
jupyter notebook --ip=0.0.0.0 --port=8888 --no-browser
````

In [1]:
!ls /home/renaissance/best_models

celestial-dragon-134  hearty-pine-137	  stilted-lake-138
chocolate-sponge-135  olive-forest-140	  sweet-eon-143
comfy-terrain-128     pious-pyramid-211   upbeat-thunder-139
crisp-sunset-141      resilient-oath-127  vibrant-oath-142
fragrant-plasma-133   sandy-blaze-130	  zesty-bird-206


In [2]:
!ls /home/renaissance/best_models/pious-pyramid-211

best_setup_e70_s39.pt  first_valid_setup_e11_s0.pt  value.pt
config.yaml	       policy.pt


In [41]:
import torch
import numpy as np
from helpers.utils import setup_kinetic_env, get_incidence_rate

In [8]:
selected_model = "pious-pyramid-211"
ckpt_dir = f"/home/renaissance/best_models/{selected_model}"
best_setup_path = f"{ckpt_dir}/policy.pt"
config_path = f"{ckpt_dir}/config.yaml"

In [9]:
cfg = OmegaConf.load(config_path)

In [10]:
best_setup_filename = [f for f in os.listdir(ckpt_dir) if "best_setup" in f][0]
best_setup_path = f"{ckpt_dir}/{best_setup_filename}"
best_setup_path

'/home/renaissance/best_models/pious-pyramid-211/best_setup_e70_s39.pt'

In [11]:
episode_str = best_setup_filename.split("_")[2]
episode = int(episode_str.split("e")[1])
step_str = best_setup_filename.split("_")[3].strip(".pt")
step = int(step_str.split("s")[1])
print(f"Best setup: episode {episode}, step {step}")

Best setup: episode 70, step 39


In [12]:
kinetic_env = setup_kinetic_env(cfg)
kinetic_env.logging_enabled = False

--------------------------------------------------
env:
  p_size: 384
  action_scale: 1
seed: 42
paths:
  names_km: data/varma_ecoli_shikki/parameter_names_km_fdp1.pkl
  output_dir: /home/renaissance/output
  met_model_name: varma_ecoli_shikki
device: cpu
logger:
  tags:
  - part2
  - max_log_std
  entity: ludekcizinsky
  project: rl-renaissance
method:
  name: ppo_refinement
  actor_lr: 0.0003
  clip_eps: 0.2
  critic_lr: 0.001
  gae_lambda: 0.98
  parameter_dim: 384
  discount_factor: 0.99
  min_max_log_std:
  - -20
  - 20
  value_loss_weight: 0.5
reward:
  eig_partition: -2.5
training:
  batch_size: 25
  num_epochs: 10
  num_episodes: 100
  max_grad_norm: 0.5
  save_trained_models: true
  max_steps_per_episode: 50
  n_eval_samples_in_episode: 50
launch_cmd: train.py method.min_max_log_std=[-20, 20] logger.tags=[part2, max_log_std]
constraints:
  max_km: 3
  min_km: -25
  ss_idx: 1712
lr_scheduler:
  name: constant

--------------------------------------------------
FYI: Loading kine

Process ForkPoolWorker-32:
Process ForkPoolWorker-12:
Process ForkPoolWorker-24:
Process ForkPoolWorker-21:
Process ForkPoolWorker-18:
Process ForkPoolWorker-23:
Process ForkPoolWorker-17:
Process ForkPoolWorker-27:
Process ForkPoolWorker-10:
Process ForkPoolWorker-4:
Process ForkPoolWorker-11:
Process ForkPoolWorker-8:
Process ForkPoolWorker-13:
Process ForkPoolWorker-20:
Process ForkPoolWorker-22:
Process ForkPoolWorker-30:
Process ForkPoolWorker-9:
Process ForkPoolWorker-1:
Process ForkPoolWorker-31:
Process ForkPoolWorker-5:
Process ForkPoolWorker-14:
Process ForkPoolWorker-28:
Process ForkPoolWorker-29:
Process ForkPoolWorker-15:
Process ForkPoolWorker-25:
Process ForkPoolWorker-3:
Traceback (most recent call last):
Process ForkPoolWorker-19:
Process ForkPoolWorker-7:
Process ForkPoolWorker-6:
Traceback (most recent call last):
Process ForkPoolWorker-16:
Process ForkPoolWorker-26:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call las

In [13]:
best_setup = torch.load(best_setup_path)

In [14]:
num_samples = 50
for i in range(10):
    incidence_rate, all_max_eigs = get_incidence_rate(kinetic_env, best_setup, num_samples)
    print(f"[Eval {i}] Incidence rate: {incidence_rate}")

Evaluating best setup:   0%|          | 0/50 [00:00<?, ?it/s]

Evaluating best setup: 100%|██████████| 50/50 [00:13<00:00,  3.61it/s]


[Eval 0] Incidence rate: 0.62


Evaluating best setup: 100%|██████████| 50/50 [00:13<00:00,  3.62it/s]


[Eval 1] Incidence rate: 0.56


...

[Eval 7] Incidence rate: 0.68


Evaluating best setup: 100%|██████████| 50/50 [00:13<00:00,  3.63it/s]


[Eval 8] Incidence rate: 0.62


Evaluating best setup: 100%|██████████| 50/50 [00:13<00:00,  3.64it/s]

[Eval 9] Incidence rate: 0.7


In [15]:
incidence_rate, all_max_eigs = get_incidence_rate(kinetic_env, best_setup, 300)

Evaluating best setup: 100%|██████████| 300/300 [01:22<00:00,  3.63it/s]


In [16]:
incidence_rate

0.6466666666666666

In [17]:
incidence_rate, all_max_eigs = get_incidence_rate(kinetic_env, best_setup, 600)

Evaluating best setup: 100%|██████████| 600/600 [02:45<00:00,  3.64it/s]


In [18]:
incidence_rate

0.6383333333333333

In [29]:
api = wandb.Api()
run = api.run("ludekcizinsky/rl-renaissance/aockrykl")

In [42]:
run.summary["best_setup/ir"] = incidence_rate
run.summary["best_setup/all_max_eigs"] = torch.tensor(all_max_eigs)
run.summary["best_setup/num_samples"] = len(all_max_eigs)
run.summary["best_setup/max_eigs_mean"] = np.mean(all_max_eigs)
run.summary["best_setup/max_eigs_min"] = np.min(all_max_eigs)
run.summary["best_setup/max_eigs_max"] = np.max(all_max_eigs)
run.summary["best_setup/max_eigs_std"] = np.std(all_max_eigs)
run.summary["best_setup/max_eigs_median"] = np.median(all_max_eigs)
run.summary["best_setup/episode"] = episode
run.summary["best_setup/step"] = step
run.summary.update()

wandb: Downloading summary data...
wandb: Uploading summary data...


In [43]:
wandb.finish()